In [ ]:
!which python
!which pip

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install pympler

In [ ]:
# !kaggle competitions download -c digit-recognizer
# !mkdir data
# !unzip digit-recognizer.zip -d ./data

In [ ]:
import sys

In [ ]:
!ls -la

In [ ]:
# Imports
from wann import WANN
from matplotlib import pyplot as plt
import pandas as pd
import json
import numpy as np
from matplotlib import pyplot as plt
from os.path import join, exists
from os import mkdir, listdir
from typing import List
from copy import copy
from random import uniform
import tensorflow as tf

Let's take a look at our data

In [ ]:
# Loading data
train_df = pd.read_csv('./data/train.csv')
y_train = train_df.pop('label')
test_df = pd.read_csv('./data/test.csv')

In [ ]:
# Show raw dataframe
train_df

In [ ]:
# Conversion: pandas dataframe to numpy array
X_train = np.array(train_df).astype('float') / 255
y_train = y_train.values
X_test = np.array(test_df).astype('float') / 255
columns = list(train_df.columns)
del train_df
del test_df

In [ ]:
X_train

In [ ]:
# wann = WANN(sensor_nodes=list(X_train.columns),
#             output_nodes=[str(i) for i in range(10)],
#             output_activation='softmax').init_randomly()
# print(json.dumps(wann.to_dict(), indent=4))

In [ ]:
# wann.draw()

In [ ]:
# img_series = X_train.iloc[7000]
# img = np.reshape(img_series.values, (28, 28))
# print(f"np.max(img) = {np.max(img)}")
# print(f"np.min(img) = {np.min(img)}")
# plt.imshow(img)
# plt.show()

In [ ]:
# wann.build_tf_graph()

In [ ]:

# result = wann.run(weight=2, sensor_data_d=img_series.to_dict(), as_numpy_array=True)
# np.argmax(result)

In [ ]:
def walk_batches(data, batchsize):
    for i in range(0, len(data), batchsize):
        yield data[i:i+batchsize]

In [ ]:
def checkpoint(wanns: List[WANN], directory: str):
    if not exists(directory):
        mkdir(directory)
    for i, wann in enumerate(wanns):
        with open(join(directory, f"WANN_{i}.json"), 'w') as f:
            json.dump(wann.to_dict(), f, indent=4)

In [ ]:
def load_checkpoint(directory: str):
    assert exists(directory), f"Directory '{directory}' doesn't exist"
    wanns = []
    for filename in listdir(directory):
        with open(join(directory, filename)) as f:
            wanns.append(WANN(input_dict=json.load(f)))
    return wanns

In [ ]:
# from pympler.tracker import SummaryTracker

# tracker = SummaryTracker()

# Training
num_wanns = 10
weights = [-2, -1, -0.5, 0.5, 1, 2]
iterations = 1000
sensor_nodes = list(columns)
output_nodes = [str(i) for i in range(10)]
batchsize = 10500
models_dir = './models'

# Initializing first wanns
wanns = [WANN(sensor_nodes, output_nodes, output_activation='softmax').init_randomly() for _ in range(num_wanns)]
# wanns = load_checkpoint(models_dir)

# X_train = X_train[:50]
# y_train = y_train[:50]

# data = list(zip(X_train, y_train))
y_train_onehot = np.zeros((len(y_train), 10), dtype=np.float32)
y_train_onehot[:, y_train] = 1.0

# Training
for iteration in range(iterations):
    
    print(f"Iteration {iteration}")
    loss = [0.] * num_wanns
    accuracy = [0.] * num_wanns
    
    checkpoint(wanns, models_dir)
    
    for i, wann in enumerate(wanns):
        
        wann.build_tf_graph()
        
        print(f"\tEvaluating WANN {i}:")
        print(f"\t\tComplexity: {wann.complexity}")
        
        # Batch walking
#         for batch in walk_batches(data, batchsize):
#             X, y = zip(*batch)
        for idx in range(0, len(X_train), batchsize):
        
            X = X_train[idx:idx+batchsize]
            y = y_train[idx:idx+batchsize]
            y_onehot = y_train_onehot[idx:idx+batchsize]
            
            # One-hot encoding label
            onehot_labels = np.zeros((len(y), 10), dtype=np.float32)
            onehot_labels[:, y] = 1.0
            
            for weight in weights:
                results = wann.run(X, weight)
                loss[i] += np.sum(np.linalg.norm(onehot_labels - results, axis=1))
                accuracy[i] += np.sum(y == np.argmax(results, axis=1))
                
        loss[i] /= len(X_train) * len(weights)
        accuracy[i] /= len(X_train) * len(weights)
        print(f"\t\tLoss: {loss[i]}")
        print(f"\t\tAccuracy: {accuracy[i]}")
        
        
    # Creating list of evaluated WANNs
    complexity = [wann.complexity for wann in wanns]
    evaluated_wanns = list(zip(wanns, loss, accuracy, complexity))

    # Choosing WANNs by mean performance
    evaluated_wanns.sort(key=lambda p: p[1])

    # Cutting WANNs with big loss (7 WANNs are left)
    del evaluated_wanns[-2:]

    # Choosing evaluation method
    evaluation_method = "accuracy" if 0 <= uniform(0, 1) < 0.9 else "complexity"

    # Evaluating remaining WANNs according to evaluation method
    if evaluation_method == "complexity":

        # Sorting WANNs by increasing complexity
        evaluated_wanns.sort(key=lambda p: p[3])

    elif evaluation_method == "accuracy":

        # Sorting WANNs by decreasing accuracy
        evaluated_wanns.sort(reverse=True, key=lambda p: p[1])

    # Cutting WANNs with small maximum performance of with big complexity
    del evaluated_wanns[-2:]

    print(f"debug: default graph len before reset: {len([n.name for n in tf.get_default_graph().as_graph_def().node])}")
    tf.reset_default_graph()
    print(f"debug: default graph len after reset: {len([n.name for n in tf.get_default_graph().as_graph_def().node])}")

    # Creating new WANNs by mutating and filling new list with them
    new_wanns = []
    for i in range(len(evaluated_wanns)):

        # Creating mutations while its not unique
        mutated_wann = copy(evaluated_wanns[i][0]).mutate()
        while mutated_wann in new_wanns or mutated_wann in wanns:
            mutated_wann = copy(evaluated_wanns[i][0]).mutate()
        new_wanns.append(mutated_wann)

        if i < 4:  # First WANNs have right to mutate twice   ODD CHECKING MIGHT BE ADD

            # Creating mutations while its not unique
            mutated_wann = copy(evaluated_wanns[i][0]).mutate()
            while mutated_wann in new_wanns or mutated_wann in wanns:
                mutated_wann = copy(evaluated_wanns[i][0]).mutate()
            new_wanns.append(mutated_wann)

    # Deleting evaluated WANNs
    for ewann, _, _, _ in evaluated_wanns:
        ewann.clear_tf_graph()
    del evaluated_wanns

    # Deleting all self.wanns
    del wanns
    wanns = new_wanns
    
#     print("CURRENT DEFAULT GRAPH".center(50, '-'))
#     print([n.name for n in tf.get_default_graph().as_graph_def().node], sep='\n')
    
#     tf.reset_default_graph()
    
#     tracker.print_diff()